<a href="https://colab.research.google.com/github/FraPet/petriglia-nlp/blob/main/trascrizione_whisper_no_diarizzazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trascrizione da file audio

Questo blocco consente di caricare un file audio, indicare manualmente i segmenti temporali da trascrivere (quelli in cui il soggetto sta effettivamente raccontando) e di ottenere un file word con proposte di tag. Questo codice è più complesso rispetto ai precedenti ed è più fragile da utilizzare ma, as usual, se si compilano le celle una alla volta senza saltarne nessuna dovrebbe essere possibile ottenere il file finale senza grosse difficoltà.

## Preparazione dell'ambiente di programmazione
Questa cella serve ad installare componenti necessari all'analisi audio. **Il tempo di esecuzione della cella è di circa 1 minuto** quindi non preoccupatevi se il programma sembra fermo.

In [1]:
%%capture
!pip install git+https://github.com/openai/whisper.git | grep -v 'already satisfied'
!pip install pydub | grep -v 'already satisfied'
!pip install pyannote.audio | grep -v 'already satisfied'

## Caricamento del file audio

Questo segmento serve a caricare il file audio nel nostro ambiente di sviluppo

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Se il file è in formato video utilizzare la prossima cella per estrarre l'audio.  In caso contrario passare direttamente alla successiva e sostituire il PATH con quello da analizzare.

In [3]:
import os, shutil
import moviepy.editor as mp

path_video = ['/content/gdrive/Shareddrives/Progetto Talkidz - Dati audio video/TALKIDZ CLINICI/RussielloG/WIN_20240223_11_03_27_Pro.mp4',
              '/content/gdrive/Shareddrives/Progetto Talkidz - Dati audio video/TALKIDZ CLINICI/RussielloG/WIN_20240223_11_24_29_Pro.mp4']


for video in path_video:
    my_clip = mp.VideoFileClip(video)
    nome_file = video.replace(".mp4", "")
    nome_file = nome_file + '.wav'

    my_clip.audio.write_audiofile(nome_file)

MoviePy - Writing audio in /content/gdrive/Shareddrives/Progetto Talkidz - Dati audio video/TALKIDZ CLINICI/RussielloG/WIN_20240223_11_03_27_Pro.wav


MoviePy - Done.
MoviePy - Writing audio in /content/gdrive/Shareddrives/Progetto Talkidz - Dati audio video/TALKIDZ CLINICI/RussielloG/WIN_20240223_11_24_29_Pro.wav


MoviePy - Done.


In [4]:
PATH_CARTELLA = 'content/gdrive/Shareddrives/Progetto Logo_Polito/PROGETTO LOGO_POLITO/VIDEO PROGETTO SOFTWARE/script_dati/trascrizione Whisper Open AI'
#PATH_WAV = '/content/gdrive/Shareddrives/Progetto Logo_Polito/PROGETTO LOGO_POLITO/VIDEO PROGETTO SOFTWARE/script_dati/trascrizione Whisper Open AI/ridotto.wav'
PATH_WAV = []
path_video = '/content/gdrive/Shareddrives/Progetto Talkidz - Dati audio video/TALKIDZ CLINICI/RussielloG'
for audio in os.listdir(path_video):
    if audio.endswith(".wav"):
#        audio = audio.replace('.mp4', '.wav')
        PATH_WAV.append(audio)

print(PATH_WAV)

['WIN_20240223_11_03_27_Pro.wav', 'WIN_20240223_11_24_29_Pro.wav']


Questo script prende in considerazione la cartella che contiene i singoli segmenti estratti dalla diarizzazione. La qualità della trascrizione sembra calare notevolmente rispetto all'analisi dell'intero segmento non diarizzato. I timestamp vanno inoltre riorganizzati perché qui ogni segment ricomincia da 0.

Nel blocco successivo c'è l'alternativa con l'analisi dell'intero segmento.

In [5]:
import whisper
from pydub import AudioSegment
import os

In [6]:
import torch
torch.cuda.empty_cache()

Provo ad utilizzare il modello large (2.88 GB), si scarica solo la prima volta che si avvia il codice


In [7]:
model = whisper.load_model("large")

100%|█████████████████████████████████████| 2.88G/2.88G [00:48<00:00, 63.6MiB/s]


In [ ]:
#segmenti = os.listdir('/content/gdrive/Shareddrives/Progetto Logo_Polito/PROGETTO LOGO_POLITO/VIDEO PROGETTO SOFTWARE/script_dati/trascrizione Whisper Open AI/diarization_files/ridotto')
trascrizione = ''
for audio in PATH_WAV:
    complete_path = f"{path_video}/{audio}"
    result = model.transcribe(complete_path, language="it")
    for element in result.items():
        print(element)

    file_ex = open(f"{os.path.basename(audio)}.txt", "w+", encoding = 'utf-8')

    for segment in result['segments']:
        start = segment['start']
        end = segment['end']
        text = segment['text']
        trascrizione += f"{text}. "

    trascrizione = trascrizione.strip()
    file_ex.write(trascrizione)
'''        if start < 60:
            print(f"{text} ({start} - {end})")
            file_ex.write(f"{text} ({start} - {end})\n")
        else:
            print(f"{text} ({round(start/60, 2)} - {round(end/60, 2)})")
            file_ex.write(f"{text} ({round(start/60, 2)} - {round(end/60, 2)})\n")'''


In [11]:
for element in result.items():
    print(element)

file_ex = open("export.txt", "w+", encoding = 'utf-8')

for segment in result['segments']:
    start = segment['start']
    end = segment['end']
    text = segment['text']

    # if start < 60:
    #     print(f"{text} ({start} - {end})")
    #     file_ex.write(f"{text} ({start} - {end})\n".strip())
    # else:
    print(f"{text} ({round(start/60, 2)} - {round(end/60, 2)})")
    file_ex.write(f"{text} ({round(start/60, 2)} - {round(end/60, 2)})\n".strip())

('text', " Pronti, questa l'abbiamo fatta, vuoi vedere la fine? Vuoi vedere l'ultima? Quando arriviamo a questa, abbiamo finito. Ok? Che bella! Ah ah! Pronti, partenza, via! Allora, una mamma è triste con una bambina perché le ha volato via il palloncino. Benissimo! Uno, quello che lavora qui, sta prendendo dei pomodori, invece una bambina sta accarezzando un asinello. Benissimo! Raccontami un altro pezzetto. Hai detto che qua ci sono tantissime cose? Che quello che lavora qui, sta prendendo dei pomodori, nel frattempo ha un ocellino sul palloncino. Bene, sì. Invece, stanno a prendere i regali e la mamma, la mamma le ha trovato un po' pazze e la bambina è felice. Per Natale. Benissimo! C'è un nonno che gli ha lanciato un osso e lui lo vuole mangiare. Benissimo! Poi sono in una... in una... in una... in una pizzeria, un po' gay. E il papà ha preso una pizza margherita, invece il bambino ha preso una piadina. Va bene, va bene. Un panino, forse. Bene. Un idraulico, però, della macchina, s